# Project of Performance and Experimentation Course

This is a practical project involving a procedure of measurement, analysis and experimentation of systems for the purpose to find possible ways to increase performance in time or space axis.

## Introduction and Topic of the project

The main idea for this project is to measure the performance between linux with its compatability layer Proton made by Valve Corporation and windows gaming; specifically for the games "Fallout: New Vegas" and "Factorio". Linux gaming has improved over the years thanks to the invention of Proton, a compatability tool created by Valve Software that allows games run on linux in a simple way eventhough the game was not done for the operating system. The protonDB page gathers reports of users of the overall stability and issues using the Proton tool. For each game it has a rating designated based on a specific criteria, the different ratings are:

- Native: the game has a linux version which does not requires a compatability tool such as Proton
- Platinum: runs perfectly out of the box
- Gold: runs perfectly after tweaks
- Silver: runs with minor issues, but generally is playable
- Bronze: runs, but often crashes or has issues preventing from playing comfortably
- Borked: game either won't start or is crucially unplayable

For the games that are gonna be used for this experiment, "Fallout: New Vegas" has a gold rating and "Factorio" has a native rating, which means these games can be runned on the linux operating system, in the case with Fallout, it might require some tweaks in the configuration files and launch options to be able to run. The ratings for both games are provided below:

- [Fallout: New Vegas](https://www.protondb.com/app/22380)
- [Factorio](https://www.protondb.com/app/427520)

The machine that's gonna be used for the experiment is a PC with the next hardware specifications:

- Processor: AMD Ryzen 7 5800x
- Memory: 32 GB DDR4
- Storage: SSD M.2 Adata Legend 800 2TB PCIe Gen4
- Graphics: Nvidia GeForce RTX 3070

As well, the software specifications comparing both operating systems are:

- Windows 11 (has a few tweaks for better perfomance and less bloat)
- Fedora 40 linux with KDE desktop environment


## Controllable factors

- Operating system version

- Desktop environment

- Active processes

- Game version

- Game resolution

- Game graphics quality

- Game texture size

- Game memory usage

- Max render threads

- Aspect Ratio

- Distance LOD

- Stage of the game: this factor means how advanced a player is in the game, for some games like Factorio, machines and processes inside the game start to stack together making the game take more resources and processing.

## Not controllable factors

- External temperature

- Scheduler algorithm

- Factory defects and general hardware damage

- Errors and crashes: for the "Fallout: New Vegas", the game publisher Bethesda (n.d.) recognizes that the game can be slightly unstable and might crash in some situations. This problems are not fault of the user, and there's nothing much to do about it that doesn't adds a modification of the game which affects it's performance in some degree.

- Resource assignment

## Studied factors

- Game resolution and aspect ratio

- Game graphics quality


## Response Variables

- Game frame rate

- Game CPU, GPU and memory usage


## Levels and Treatments

### Treatments for Fallout: New Vegas

For the aspect ratio, there are only 2 options which is the standard 4:3 or 5:4, and the widescreen that is 16:9. The resolution has many options, but because my monitor has its limit to 1920x1080, that will be the maximum resolution available, and the rest will be choosen based on how frequent it's to find someone with a monitor with that exact resolution: 1440x810 and 1680x945. Then the quality of the game graphics there's only 4: Low, medium, high and ultra, because this game is not that heavy in graphic processing, we can only use the high and ultra quality. In total, there will be 6 treatments with 2 levels each.

### Treatments for Factorio

The game doesn't gives a way to set up a resolution, that's because it takes the whole screen, which in my case is 1920x1080 resolution. However, it gives more options to the player for texture quality and graphic processing. These are the sprite resolution that can be set to high or normal, in this case it will not change and be static to high. The max number of threads that for now it uses as default 8 threads to render the game; this is the most important variable because as the game progresses it will require more processing, for the purpose of this experiment, the set of number of threads to use will be 4, 8 and 16. The video memory usage which can be set to all, high, medium and low; for the purpose of this investigation it will only be set to all and high. And Texture compression that can be set to high quality and low quality, as well this won't be changed and will remain in high quality compression. This gives a total of 6 treatments with 3 levels each.


## References

ProtonDB. (n.d.) ProtonDB main page. https://www.protondb.com/

Bethesda Support. (n.d.). What do I do if Fallout: New Vegas is crashing on my PC?. https://help.bethesda.net/#en/answer/35756

## Credits

Done by Daniel Mora Rodriguez (C25200), student of the Universidad de Costa Rica, Escuela de Ciencias de la Computacion e Informatica for the second semester of  2024.